# install and import libraries

In [ ]:
!pip install -q bitsandbytes==0.43.1 datasets==2.19.0 peft==0.10.0 trl==0.8.6 accelerate==0.29.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requir

In [ ]:
import os
import torch
import re
import json
import gdown
from datasets import Dataset
import pandas as pd
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from tqdm.auto import tqdm
from trl import DPOTrainer

# load the pre-trained model

In [ ]:
model_name="MediaTek-Research/Breeze-7B-Instruct-v0_1"
model_path="/content/drive/MyDrive/Colab Notebooks/GenAI/Breeze"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# quantify
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_use_triton=True
)

# load model parameter
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    cache_dir=model_path,
    quantization_config=quantization_config

)
# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=model_path,
    quantization_config=quantization_config
)

# set decoding stategy
# random sampling
max_len=128
generation_config = GenerationConfig(
    max_length=max_len,
    do_sample=True,
    temperature=0.9,
    top_p=0.5,
    no_repeat_ngram_size=3,
    pad_token_id=2,
)

Unused kwargs: ['bnb_4bit_use_triton']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:991: UserWarning: Not enough free disk space to download the file. The expected file size is: 9987.00 MB. The target location /content/drive/MyDrive/Colab Notebooks/GenAI/Breeze/models--MediaTek-Research--Breeze-7B-Instruct-v0_1/blobs only has 4788.10 MB free disk space

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# load data

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/GenAI

/content/drive/MyDrive/Colab Notebooks/GenAI


In [ ]:
data_file='/content/drive/MyDrive/Colab Notebooks/GenAI'
train_data_file='/content/drive/MyDrive/Colab Notebooks/GenAI/GenAI_hw6_dataset/labelled_data.json'
test_data_file='/content/drive/MyDrive/Colab Notebooks/GenAI/GenAI_hw6_dataset/test_prompt.json'


In [ ]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

Cloning into 'GenAI_hw6_dataset'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), 4.06 KiB | 693.00 KiB/s, done.


In [ ]:
with open(test_data_file,'r',encoding='utf-8') as f:
  test_datas=json.load(f)

with open(train_data_file,'r',encoding='utf-8') as f:
  train_datas=json.load(f)

support_ratio=0
train_datas_size=len(train_datas)
support_size=int(support_ratio*train_datas_size)
prompt_list=[]
position_list=[]
preferred_list=[]
non_preferred_list=[]

prompt_list=[data['prompt'] for data in train_datas]
position_list=['support']*support_size+['oppose']*(train_datas_size-support_size)
preferred_list=[data['support'] for data in train_datas[:support_size]]+[data['oppose'] for data in train_datas[support_size:]]
non_preferred_list=[data['oppose'] for data in train_datas[:support_size]]+[data['support'] for data in train_datas[support_size:]]
train_dict={
  'prompt':prompt_list,
  'position':position_list,
  'chosen':preferred_list,
  'rejected':non_preferred_list
}

train_datasets=Dataset.from_dict(train_dict)
pd.DataFrame(train_datasets)

,prompt,position,chosen,rejected
0,日本動漫真人化是否有損原作形象？,oppose,真人化可能無法完美呈現動畫中的獨特風格，損害原作形象。,真人化能夠呈現更真實的角色形象，提升原作魅力。
1,真人化是否能夠擴大動漫在全球的影響力？,oppose,真人化可能失去動漫的獨特風格，限制影響力擴大。,真人化能夠讓更多非動漫迷接觸作品，擴大影響力。
2,真人化是否能夠吸引新觀眾？,oppose,真人化可能讓原本的動漫迷感到失望，無法吸引新觀眾。,真人化能夠吸引不熟悉動漫的觀眾，擴大受眾。
3,真人化是否能夠保留原作故事情節的精髓？,oppose,真人化可能因為改編而失去原作故事的深度與精髓。,真人化有機會更深入挖掘原作故事，保留精髓。
4,真人化是否能夠提升動漫產業的商業價值？,oppose,真人化可能讓觀眾對原作失去興趣，影響產業價值。,真人化能夠開拓更多商業機會，提升產業價值。
5,真人化是否能夠保持原作的文化特色？,oppose,真人化可能因為文化差異而失去原作獨有的文化魅力。,真人化可以透過場景、服裝等元素保留文化特色。
6,真人化是否能夠挑戰技術上的新突破？,oppose,真人化可能因為技術限制而無法達到動畫中的視覺效果。,真人化促使技術創新，挑戰視覺效果上的新高度。
7,真人化是否會受到演員選擇的爭議？,oppose,演員選擇可能引起爭議，觀眾難以接受角色塑造。,演員選擇可因應市場需求，不必受限於動畫形象。
8,真人化是否能夠提高動漫的社會認同度？,oppose,真人化可能因為劇情改編而無法贏得社會認同。,真人化有機會讓更多人接受動漫，提高社會認同度。
9,真人化是否能夠保留原作角色的個性特色？,oppose,真人化可能因演員演技或導演選擇而失去角色的原有特色。,真人化可以透過演員表現保留角色的個性特色。


# test the output of pre-trained model

In [ ]:
prompt_template="""
[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
{input}
[/INST]
"""
from tqdm import tqdm
import time

for data in tqdm(test_datas):
  input_text=prompt_template.format(input=data['prompt'])
  input_ids=tokenizer(input_text, return_tensors="pt").input_ids.cuda()
  response=model.generate(
      input_ids=input_ids,
      max_length=max_len,
      do_sample=generation_config.do_sample,
      temperature=generation_config.temperature,
      num_beams=generation_config.num_beams,
      top_p=generation_config.top_p,
      no_repeat_ngram_size=generation_config.no_repeat_ngram_size,
      pad_token_id=generation_config.pad_token_id
  )
  generated_text = tokenizer.decode(response[0], skip_special_tokens=True)

  # 打印生成的文本
  print(generated_text)
  print()

 10%|█         | 1/10 [00:08<01:17,  8.65s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否能改善日本漫畫的全球可及性？
[/INST]
真人化的可能性，或許能提高日本漫畫在全球的可及性和接受度。真人化指的是將漫畫人物以真人演員演出的電影或電視劇。真人化的作品能吸引更多的觀眾，尤其是那些不習慣閱讀漫畫或對漫畫人物有特殊情感的觀眾。

然而，真人化並非萬能的解決辦法。成功的真人化需要多



 20%|██        | 2/10 [00:16<01:04,  8.09s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化如何影響年輕一代對日本漫畫的看法？
[/INST]
真人化的影響使年輕一代的看法更加多元化。一些年輕觀眾可能更喜歡真人版日本漫畫，因為它提供了更接近現實的故事和角色。真人版可以吸引新觀眾，特別是那些對動漫文化尚未熟悉的年輕人。然而，也有一些人認為真人版無法完全捕捉動漫的魅力，而傾向於傳統的動畫作品。總的來說，



 30%|███       | 3/10 [00:23<00:53,  7.69s/it]

...


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否能夠促進日本漫畫的創新和多樣性？
[/INST]
真人化的可能性，在日本漫畫（動漫）的世界中，確實可以促進創新和多元性。真人化指的是將漫畫人物以真人的方式呈現，例如電影、電視劇、舞台劇等。真人化的過程中，可能會引入新的故事、角色、視覺效果，以及融合其他文化元素，進而促進動漫的創新。

在日本，動漫



 90%|█████████ | 9/10 [01:07<00:07,  7.40s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否有助於擴大動漫文化的市場份額？
[/INST]
真人化的動漫角色可以有助於拓展動漫文化在市場上的份額。真人化指的是動漫角色以真人演員或動畫人物的形式在影視劇、電影、廣告等不同的媒體中呈現，以吸引更多的觀眾群。真人化的效果可以提高動漫角色的知名度，吸引更多人認識動漫文化，從而促進動漫文化市場的拓展。

然而，真人



100%|██████████| 10/10 [01:14<00:00,  7.40s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否有助於提高日本漫畫在全球的競爭力？
[/INST]
真人化的日本漫畫是否有助提高在全球競爭力，這問題需要深入分析。真人化指的是將漫畫人物以真人演員演出的電影或電視劇。真人化的優點是可以吸引更多的觀眾，尤其是那些不習慣閱讀漫畫的觀眾。真人版《鬼太郎》和《名偵探柯南》等作品在全球獲得了成功，證明了真人



# DPO算法微调模型

In [ ]:
num_epoch = 10

In [ ]:
dpo_trainer = DPOTrainer(
    model,
    args=TrainingArguments(
      output_dir='./',
      per_device_train_batch_size=1,
      num_train_epochs=num_epoch,
      gradient_accumulation_steps=8,
      gradient_checkpointing=False,
      learning_rate=2e-4,
      optim="paged_adamw_8bit",
      logging_steps = 1,
      warmup_ratio = 0.1,
      report_to = 'none'
    ),
    beta=0.1,
    train_dataset=train_datasets,
    tokenizer=tokenizer,
    peft_config=LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM"
    )
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
1,0.693100
2,0.693100
3,0.668500
4,0.627100


In [ ]:
for data in tqdm(test_datas):
  input_text=prompt_template.format(input=data['prompt'])
  input_ids=tokenizer(input_text, return_tensors="pt").input_ids.cuda()
  response=model.generate(
      input_ids=input_ids,
      max_length=max_len,
      do_sample=generation_config.do_sample,
      temperature=generation_config.temperature,
      num_beams=generation_config.num_beams,
      top_p=generation_config.top_p,
      no_repeat_ngram_size=generation_config.no_repeat_ngram_size,
      pad_token_id=generation_config.pad_token_id
  )
  generated_text = tokenizer.decode(response[0], skip_special_tokens=True)

  # 打印生成的文本
  print(generated_text)
  print()

 10%|█         | 1/10 [00:06<00:57,  6.37s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否能改善日本漫畫的全球可及性？
[/INST]
真人化的可能無法解決日本漫畫在全球的可及性的問題，因為文化差異和市場偏好可能仍存在。而為了因應真人化可能失敗的風險，日本漫畫可能須考慮其他策略，例如加強海外行銷或調整故事內容以符合全球觀眾胃口。



 20%|██        | 2/10 [00:13<00:54,  6.86s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化如何影響年輕一代對日本漫畫的看法？
[/INST]
真人化的可能影響可能會損害年輕一代對於日本漫畫之看法，因他們可能因無法接受真人化而喪失對原漫畫文化的興趣。此外，這種改變可能忽略日本漫畫獨特的風格與元素，導致觀眾無法享受原作所帶來的體驗。



 30%|███       | 3/10 [00:16<00:36,  5.18s/it]

...


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否能夠促進日本漫畫的創新和多樣性？
[/INST]
真人化的可能無法促進日本動畫的創新與多樣性，反而可能因商業考量而限縮創作自由，導致產業缺乏新穎想法和多元化。



 90%|█████████ | 9/10 [00:51<00:05,  5.54s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否有助於擴大動漫文化的市場份額？
[/INST]
真人化的可能可能因缺乏動漫文化獨特性而無法吸引觀眾，而影響動漫文化在市場上的競爭力。此情形可能導致動漫文化無法獲得足夠關注與支持，而無法持續發展。



100%|██████████| 10/10 [00:58<00:00,  5.83s/it]


[INST] <<SYS>>
You are a helpful assistant and good at writing Tang poem.
<</SYS>>
真人化是否有助於提高日本漫畫在全球的競爭力？
[/INST]
真人化的可能性可能會因為文化差異而無法有效地傳達原著精神，而可能導致日本漫畫在其全球競爭中失去優勢。此外，可能因觀眾期待不符而導致負面評價，進而影響其在全球市場的接受度。因此，真人化可能無法有效提升日本漫畫的競爭力道。

